In [2]:
import os
import moviepy.editor as mp
import librosa
import librosa.display
import IPython.display
from PIL import Image
import cv2
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline

In [ ]:
# wav 파일 만들기

video_list = os.listdir('video_train')
video_dir = './video_train/'
i = 0

for file in video_list:
    os.mkdir('./sound_train/'+ file[:-4] + '_sound')
    length = int((cv2.VideoCapture(video_dir + file).get(cv2.CAP_PROP_FRAME_COUNT))/29.98)
    while True:
        if i < length: # 아마 캡쳐 이미지 수가 음성 파일 갯수와 다를 것. 음성 파일 갯수를 캡쳐 이미지 수에 맞추기.
            clip = mp.VideoFileClip(video_dir + file)
            clip = clip.subclip(t_start=(i), t_end=(i+0.5))
            clip.audio.write_audiofile(('./sound_train/' + file[:-4] + '_sound/' + file[:-4] + '_sound_%0.4f.wav') % i)
            i += 0.5
            clip.close()
        else:
            i = 0
            break

In [ ]:
# mfcc 만들기

sound_list = os.listdir('sound_train')

for game in sound_list:
    
    sound_dir = os.listdir('./sound_train/' + game)
    os.mkdir('./train_mfcc_original/'+game)
    os.mkdir('./traint_mfcc/'+game)
        
    for i in range(len(sound_dir)):
        
        y, sr = librosa.load('./sound_train/' + game +'/'+ sound_dir[i])
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        plt.figure(figsize=(3,3))
        librosa.display.specshow(mfcc, sr=sr)
        fig = plt.gcf()
        fig.savefig('./train_mfcc_original/'+game+'/'+ game[:-5] + 'train_mfcc_%04d.jpg' % i, transparent=True)
        image = Image.open('./train_mfcc_original/'+game+'/'+ game[:-5] + 'train_mfcc_%04d.jpg' % i)
        resize_image = image.resize((100,100))
        resize_image.save('./train_mfcc/'+game+'/'+ game[:-5] + 'train_mfcc_%04d.jpg' % i)
        
    print (game[:-5] + "Done!")

In [ ]:
# 파일 이름 바꾸기 ('train_mfcc_%d.jpg'라는 형식으로 저장해버려 라벨링에 문제 생겼기 때문...)

import os

file_path = '옮겨담을 mfcc & sound 파일들/train_mfcc'
file_names = os.listdir(file_path)

for file in file_names:
    mfcc_list = os.listdir(file_path+'/'+file)
    i = 0
    for name in mfcc_list:
        os.rename(file_path+'/'+file+'/'+name, file_path+'/'+file+'/'+file[:-5]+'train_mfcc_%04d.jpg' % i)
        i += 1

In [37]:
# 여러 개의 폴더로 나뉘어져 존재하는 mfcc들을 한 군데에 모으기

# train mfcc와 train image의 수가 조금 다르다. 그러나 라벨링 csv 파일은 image 기준으로 되어있고 mfcc도 똑같은 라벨링 파일을 쓴다.
# 그러므로 train mfcc에서는 train image와 짝이 되는 것들만 뽑아서 학습에 쓰인다.

import shutil
file_path = '옮겨담을 mfcc & sound 파일들/train_mfcc'
folder = os.listdir(file_path)

for file in folder:
    mfcc_list = os.listdir('train_mfcc_temp/'+file)
    for mfcc in mfcc_list:
        shutil.copy('train_mfcc_temp/'+file+'/'+mfcc, 'train_mfcc')